In [1]:
## TESTS START FROM LINE 2156 (tomohara-aliases.bash)

## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-py-commands

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

## TAKES EXTREMELY LONG TIME TO SOURCE THE MAIN BASH FILEII
# source tomohara-aliases.bash

alias | wc -l

In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3443)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3443

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

In [7]:
export PYTHON_CMD="/usr/bin/time python -u"
export PYTHON="$NICE $PYTHON_CMD"
export PYTHONPATH="$HOME/python:$PYTHONPATH"

In [8]:
# add-python-path(pkg-dir): add PKG-DIR to PATH and PARENT to PYTHONPATH
## HACK: make sure Mezcla/mezcla resolves before python/mezcla
## OLD: export PYTHONPATH="$HOME/python/Mezcla:$PYTHONPATH"
function add-python-path () {
    local package_path="$1"
    local parent_path=$(realpath "$package_path/..")
    # add package to path (e.g., $HOME/python/Mezcla/mezcla)
    prepend-path "$package_path"
    # add parent to python-path spec (e.g., $HOME/python/Mezcla)
    export PYTHONPATH="$parent_path:$PYTHONPATH"
}
# EX: mezcla-devel; which system.py | grep -i Mezcla-main => ""
# note: mezcla-devel should be stable version of mezcla-tom
## OLD: alias mezcla-devel='add-python-path $HOME/programs/python/mezcla-tom/mezcla'
alias mezcla-devel='add-python-path $HOME/python/Mezcla/mezcla'
alias mezcla-main='add-python-path $HOME/python/Mezcla-main/mezcla'
alias mezcla-tom='add-python-path $HOME/python/Mezcla-tom/mezcla'
# Add mezcla-devel unless another version in path
if [[ ! "$PATH" =~ mezcla ]]; then
    ## OLD: mezcla-devel
    ## TODO: echo "Warning: mezcla not in PATH"
    true
fi

In [9]:
## FUNCTIONS REQUIRED FOR SMOOTH RUNNING OF THE TESTS
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }
# Path settings
# TODO: define a function for removing duplicates from the PATH while
# preserving the order
function show-path-dir () { (echo "${1}:"; printenv "$1" | perl -pe "s/:/\n/g;") | $PAGER; }
alias show-path='show-path-dir PATH'
# append-path(path): appends PATH to environment variable unless already there
## TODO: function in-path { local path=$(tr ":" "\n" | $GREP "^$1$$"); return ($path != ""); }
# TODO: add force argument to ensure last (or first)
function append-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="${PATH}:$1"; fi }
function prepend-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="$1:${PATH}"; fi }
# TODO: rework append-/prepend-path and python variants via generic helper
function append-python-path () { export PYTHONPATH=${PYTHONPATH}:"$1"; }
function prepend-python-path () { export PYTHONPATH="$1":${PYTHONPATH}; }
 

In [10]:
# add-python-path $temp_dir
add-python-path $temp_dir

In [11]:
alias ps-python-full='ps-all python'
# note: excludes ipython and known system-related python scripts
alias ps-python='ps-python-full | $EGREP -iv "(screenlet|ipython|egrep|update-manager|software-properties|networkd-dispatcher)"'
alias show-python-path='show-path-dir PYTHONPATH'

In [12]:
## ERROR 1 - process ID list syntax error

# $ ps-python-full
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).

In [13]:
## ERROR 2 - bash : command not found and process ID list syntax error
# $ ps-python
# | bash: -iv: command not found
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).


In [14]:
## show-python-path DISPLAYS ALL THE AVAILABLE PATHS OF PYTHON
show-python-path

PYTHONPATH:
/tmp/test-py-commands
/home/aveey/python



In [15]:
# Python-lint filtering
# python-lint-full(filename): complete output from pylint, with caret-based
# context indicators retained by substiting carriage return for newline.
# python-lint-work(filename): pylint with moderate filtering
# python-lint(filename): pylint with usual filtering
# TODO: specify exclusion types in pylint command line (e.g., invalid-name)
# example: "C:674, 0: Invalid constant name "term_freq" (invalid-name)"
# example: "run_ner.py:413:0: C0330: Wrong continued indentation (add 8 spaces).
#                   'B-DATE', 'I-DATE', 'B-DOCTOR', 'I-DOCTOR', 'B-LOCATION', 'I-LOCATION', 'B-AGE', 'I-AGE',
#                   ^       | (bad-continuation)
# TODO: make Mercurial root-to-python-path hack optional
# TODO: add similar aliases for pep8 and pyflakes
# TODO: handle continutations of statements without indentation:
#   ner_eval/ner_detokenize.py:11:45: C0326: Exactly one space required after comma
#   parser.add_argument('--output_file', type=str,  help='')
#                                      ^ (bad-whitespace)

function python-lint-full() { 
    local root=$(hg root 2> /dev/null);
    ## TODO: --persistent=n (to avoid caching)
    PYTHONPATH="$root:.:$PYTHONPATH" $NICE pylint "$@" | perl -00 -ne 'while (/(\n\S+:\s*\d+[^\n]+)\n( +)/) { s/(\n\S+:\s*\d+[^\n]+)\n( +)/$1\r$2/mg; } print("$_");' 2>&1 | $PAGER;
}

In [16]:
# python-lint-full (scores the code on the basis of effectiveness)
python-lint-full $BIN_DIR/tests/batspp.py


************* Module batspp
/home/aveey/tom-project/shell-scripts/tests/batspp.py:378:0: R0022: Useless option value for 'disable', 'no-self-use' was moved to an optional extension, see https://pylint.pycqa.org/en/latest/whatsnew/2/2.14/summary.html#removed-checkers. (useless-option-value)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:13:0: C0301: Line too long (129/100) (line-too-long)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:14:0: C0301: Line too long (188/100) (line-too-long)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:20:0: C0301: Line too long (175/100) (line-too-long)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:84:0: C0301: Line too long (101/100) (line-too-long)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:115:0: C0301: Line too long (108/100) (line-too-long)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:144:17: C0303: Trailing whitespace (trailing-whitespace)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:156:10: 

/home/aveey/tom-project/shell-scripts/tests/batspp.py:466:9: W0511: TODO: raise NotImplementedError() (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:499:9: W0511: TODO: make this optional for expected output field (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:575:9: W0511: # TODO: temp_dir = Main.temp_base (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:631:9: W0511: TODO: use an external file (as the @here might fail if the example uses << as well) (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:655:13: W0511: # TODO: self._setup_funct = None (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:672:73: W0511: TODO: fix (Bruno, what is for?) (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:673:73: W0511: TODO: fix (ditto) (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:771:13: W0511: # TODO: (fixme)
/home/aveey/tom-project/shell-scripts/tests/batspp.py:784:9: W0511: # TODO: (fixme)
/home/avee

In [17]:
# Notes:
# - filters out context in addition to warning proper, as in following:
#    ex: C:244, 0: Exactly one space required after assignment
#    SKIP_ADS =  (IS_GI_JOB_SEARCH or system.getenv_bool("SKIP_ADS", False))
#             ^ (bad-whitespace)
# - filters out other extraneous output
#   ex: Your code has been rated at 7.43/10 ...
#   ex: No config file found ...
# - the following has two regex: *modify the first* to add more conditions to ignore; the second is just for the extraneous pylint output
function python-lint-work() { python-lint-full "$@" 2>&1 | $EGREP -v '\((bad-continuation|bad-option-value|fixme|invalid-name|locally-disabled|too-few-public-methods|too-many-\S+|trailing-whitespace|star-args|unnecessary-pass)\)' | $EGREP -v '^(([A-Z]:[0-9]+)|(Your code has been rated)|(No config file found)|(\-\-\-\-\-))' | $PAGER; }
# TODO: rename as python-lint-tpo for clarity (and make python-lint as alias for it)
function python-lint() { python-lint-work "$@" 2>&1 | $EGREP -v '(Exactly one space required)|\((bad-continuation|bad-whitespace|bad-indentation|bare-except|c-extension-no-member|consider-using-enumerate|consider-using-with|global-statement|global-variable-not-assigned|keyword-arg-before-vararg|len-as-condition|line-too-long|logging-not-lazy|misplaced-comparison-constant|missing-final-newline|redefined-variable-type|redundant-keyword-arg|superfluous-parens|too-many-arguments|too-many-instance-attributes|trailing-newlines|useless-\S+|wrong-import-order|wrong-import-position)\)' | $PAGER; }


In [18]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint-work $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [19]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [20]:
# run-python-lint-batched([file_spec="*.py"]: Run python-lint in batch mode over
# files in FILE_SPEC, placing results in pylint/<today>.
#
function get-python-lint-dir () {
    local python_version_major=$(pylint --version 2>&1 | extract_matches.perl "Python (\d)")
    local affix="py${python_version_major}"
    local out_dir="_pylint/$(todays-date)-$affix"
    echo "$out_dir"
}
#
function run-python-lint-batched () {
    # TODO: support files with embedded spaces
    local file_spec="$@"
    if [ "$file_spec" = "" ]; then file_spec="*.py"; fi

    # Create output directory if needed
    local out_dir=$(get-python-lint-dir)
    mkdir -p "$out_dir"

    # Run pylint and pipe top section into less
    (for f in $($LS $file_spec); do
         # HACK: uses basename of parent prefix if invoked with path
         local b=$(basename "$f")
         local pre=""
	 # Note: uses directory name as prefix if file not in current dir
         if [[ $f =~ / ]]; then pre="$(basename $(dirname "$f"))-"; fi
         DEBUG_LEVEL=5 python-lint "$f" >| "$out_dir/$pre$b".log 2>&1
         head "$out_dir/$pre$b".log
     done) >| "$out_dir/summary.log"
    less -p '^\** Module' "$out_dir/summary.log";
}

In [21]:
# (REQUIRED BY run-python-lint-batched)

function downcase-stdin { perl -pe "use open ':std', ':encoding(UTF-8)'; s/.*/\L$&/;"; }
function downcase-text() { echo "$@" | downcase-stdin; }
# todays-date(): outputs date in format DDmmmYY (e.g., 22Apr20)
## OLD: function todays-date() { date '+%d%b%y' | perl -pe 's/.*/\L$&/;'; }
## TODO: drop leading digits in day of month
## NOTE: keep in synch with common.perl get_file_ddmmmyy and .emacs edit-adhoc-notes-file
function todays-date() { date '+%d%b%y' | downcase-stdin; }


In [22]:
# ERROR: Pattern not found  (press RETURN)
# run-python-lint-batched ./print.py 

In [23]:
# python-import-path(module): find path for package directory of MODULE
# Note: this checks output via module initialization output shown with python -v
# ex: /usr/local/misc/programs/anaconda3/lib/python3.8/site-packages/sklearn/__pycache__/__init__.cpython-38.pyc matches /usr/local/misc/programs/anaconda3/lib/python3.8/site-packages/sklearn/__init__.py

function python-import-path-all() { local module="$1"; python -u -v -c "import $module" 2>&1; }
function python-import-path-full() { local module="$1"; python-import-path-all "$@" | extract_matches.perl "((matches (.*\W$module[^/]*[/\.][^/]*))|ModuleNotFoundError)"; }
function python-import-path() { python-import-path-full "$@" | head -1; }


In [24]:
# RETURNS A LIST OF PYTHON MODULES
pip3 freeze

asttokens==2.0.5
Babel==2.10.1
backcall==0.2.0
bash_kernel @ git+https://github.com/kdm9/bash_kernel.git@a4c81c9f35926b7625b11e4c016c5e67edac522f
batspp @ file:///home/aveey/tom-project/shell-scripts/batspp
brotlipy==0.7.0
certifi==2021.10.8
cffi @ file:///opt/conda/conda-bld/cffi_1642701102775/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
click==8.1.3
colorama @ file:///tmp/build/80754af9/colorama_1607707115595/work
conda==4.12.0
conda-content-trust @ file:///tmp/build/80754af9/conda-content-trust_1617045594566/work
conda-package-handling @ file:///tmp/build/80754af9/conda-package-handling_1649105784853/work
cryptography @ file:///tmp/build/80754af9/cryptography_1639414572950/work
debugpy==1.6.0
decorator==5.1.1
entrypoints==0.4
executing==0.8.3
ExifRead==3.0.0
filetype==1.0.13
Flask==2.0.1
idna @ file:///tmp/build/80754af9/idna_1637925883363/work
importlib-metadata==4.11.4
inifile==0.4.1
ipykernel==6.15.0
ipython==8.4.0
itsdangerous==2.1.2

In [25]:
# conda MODULE IS USED FOR TEST

python-import-path 'conda'
linebr
python-import-path-full 'conda'
linebr
python-import-path-all 'conda' | grep 'conda'

matches /home/aveey/miniconda3/lib/python3.9/site-packages/conda/__init__.py
--------------------------------------------------------------------------------
matches /home/aveey/miniconda3/lib/python3.9/site-packages/conda/__init__.py
matches /home/aveey/miniconda3/lib/python3.9/site-packages/conda/common
matches /home/aveey/miniconda3/lib/python3.9/site-packages/conda/common
--------------------------------------------------------------------------------
# /home/aveey/miniconda3/lib/python3.9/encodings/__pycache__/__init__.cpython-39.pyc matches /home/aveey/miniconda3/lib/python3.9/encodings/__init__.py
# code object from '/home/aveey/miniconda3/lib/python3.9/encodings/__pycache__/__init__.cpython-39.pyc'
# /home/aveey/miniconda3/lib/python3.9/__pycache__/codecs.cpython-39.pyc matches /home/aveey/miniconda3/lib/python3.9/codecs.py
# code object from '/home/aveey/miniconda3/lib/python3.9/__pycache__/codecs.cpython-39.pyc'
# /home/aveey/miniconda3/lib/python3.9/encodings/__pycache__/ali

# /home/aveey/miniconda3/lib/python3.9/__pycache__/copyreg.cpython-39.pyc matches /home/aveey/miniconda3/lib/python3.9/copyreg.py
# code object from '/home/aveey/miniconda3/lib/python3.9/__pycache__/copyreg.cpython-39.pyc'
# /home/aveey/miniconda3/lib/python3.9/json/__pycache__/scanner.cpython-39.pyc matches /home/aveey/miniconda3/lib/python3.9/json/scanner.py
# code object from '/home/aveey/miniconda3/lib/python3.9/json/__pycache__/scanner.cpython-39.pyc'
# extension module '_json' loaded from '/home/aveey/miniconda3/lib/python3.9/lib-dynload/_json.cpython-39-x86_64-linux-gnu.so'
# extension module '_json' executed from '/home/aveey/miniconda3/lib/python3.9/lib-dynload/_json.cpython-39-x86_64-linux-gnu.so'
# /home/aveey/miniconda3/lib/python3.9/json/__pycache__/encoder.cpython-39.pyc matches /home/aveey/miniconda3/lib/python3.9/json/encoder.py
# code object from '/home/aveey/miniconda3/lib/python3.9/json/__pycache__/encoder.cpython-39.pyc'
# /home/aveey/miniconda3/lib/python3.9/site-p

In [26]:
# note: gotta hate python!
function python-module-version-full { local module="$1"; python -c "import $module; print([v for v in [getattr($module, a, '') for a in '__VERSION__ VERSION __version__ version'.split()] if v][0])"; }
# TODO: check-error if no value returned
function python-module-version { python-module-version-full "$@" 2> /dev/null; }
function python-package-members() { local package="$1"; python -c "import $package; print(dir($package));"; }
#
alias python-setup-install='log=setup.log;  rename-with-file-date $log;  uname -a > $log;  python setup.py install --record installed-files.list >> $log 2>&1;  ltc $log'
# TODO: add -v (the xargs usage seems to block it)
alias python-uninstall-setup='cat installed-files.list | xargs /bin/rm -vi; rename_files.perl -regex ^ un installed-files.list'


In [27]:
# MODULE USED: mezcla
python-module-version-full mezcla
linebr
python-module-version mezcla
linebr
python-package-members mezcla
linebr

1.3.0
--------------------------------------------------------------------------------
1.3.0
--------------------------------------------------------------------------------
['PYTHON3_PLUS', 'TL', '__VERSION__', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'debug', 'gh', 'glue_helpers', 'mezcla', 'sys', 'sys_version_info_hack', 'system', 'tpo_common']
--------------------------------------------------------------------------------


In [28]:
## rename-with-file-date REQUIRED
function rename-with-file-date() {
    ## DEBUG: set -o xtrace
    local f new_f
    local move_command="move"
    if [ "$1" = "--copy" ]; then
        ## TODO: move_command="copy"
        move_command="command cp --interactive --verbose --preserve"
        shift
    fi
    for f in "$@"; do
        ## DEBUG: echo "f=$f"
        if [ -e "$f" ]; then
           new_f=$(get-free-filename "$f".$(date --reference="$f" '+%d%b%y') ".")
           ## DEBUG: echo
           eval "$move_command" "$f" "$new_f";
        fi
    done;
    ## DEBUG: set - -o xtrace
}

function get-free-filename() {
    local base="$1"
    local sep="$2"
    local L=1
    local filename="$base"
    ## DEBUG: local -p
    while [ -e "$filename" ]; do
        let L++
        filename="$base$sep$L"
    done;
    ## DEBUG: local -p
    echo "$filename"
}

alias move='mv'
export PAGER=less
export PAGER_CHOPPED="less -S"
export PAGER_NOEXIT="less -+F"
function zless () { zcat "$@" | $PAGER; }
# 
# zhead(file, head-opts)
function zhead () {
    local file="$1"
    shift
    zcat "$file" | head "$@"
}
alias less-='$PAGER_NOEXIT'
alias less-clipped='$PAGER_NOEXIT -S'
alias less-tail='$PAGER_NOEXIT +G'
alias less-tail-clipped='$PAGER_NOEXIT +G -S'
alias ltc=less-tail-clipped


In [29]:
## REQUIRES setup.py (TBC)

#python-setup-install
linebr
#python-uninstall-setup

--------------------------------------------------------------------------------


In [30]:
# ipython(): overrides ipython command to set xterm title
alias set-xterm-window='set_xterm_title.bash'

function ipython() { 
    local ipython=$(which ipython)
    if [ "$ipython" = "" ]; then echo "Error: install ipython first"; return; fi
    set-xterm-window "ipython [$PWD]"
    $ipython "$@"
}

# python-trace(script, arg, ...): Run python SCRIPT with statement tracing
function python-trace {
    local script="$1"
    shift
    $PYTHON -m trace --trace $(which "$script") "$@"
    }


In [31]:
# ipython COMMAND SEEMS TO WORK
ipython -h

/home/aveey/bin/set_xterm_title.bash: line 239: /_set_xterm_title.12186.full.list: Permission denied
/home/aveey/bin/set_xterm_title.bash: line 240: /_set_xterm_title.12186.icon.list: Permission denied
]1;ipython [/tmp/test-py-commands/test-3443]]2;ipython [/tmp/test-py-commands/test-3443]=========
 IPython

Tools for Interactive Computing in Python

    A Python shell with automatic history (input and output), dynamic object
    introspection, easier configuration, command completion, access to the
    system shell and more.  IPython can also be embedded in running programs.

Usage

    ipython [subcommand] [options] [-c cmd | -m mod | file] [--] [arg] ...

    If invoked with no options, it executes the file and exits, passing the
    remaining arguments to the script, just as if you had specified the same
    command with python. You may need to specify `--` before args to be passed
    to the script, to prevent IPython from attempting to parse them. If you
    specify the optio

    Equivalent to: [--TerminalIPythonApp.force_interact=True]
--log-level=<Enum>
    Set the log level by value or name.
    Choices: any of [0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL']
    Default: 30
    Equivalent to: [--Application.log_level]
--profile-dir=<Unicode>
    Set the profile location directly. This overrides the logic used by the
            `profile` option.
    Default: ''
    Equivalent to: [--ProfileDir.location]
--profile=<Unicode>
    The IPython profile to use.
    Default: 'default'
    Equivalent to: [--BaseIPythonApplication.profile]
--ipython-dir=<Unicode>
    The name of the IPython directory. This directory is used for logging
    configuration (through profiles), history storage, etc. The default is
    usually $HOME/.ipython. This option can also be specified through the
    environment variable IPYTHONDIR.
    Default: ''
    Equivalent to: [--BaseIPythonApplication.ipython_dir]
--config=<Unicode>
    Path to an extra config file 

In [33]:
python-trace l $BIN_DIR/tests/ipynb_to_batspp.py #(need some help to test it properly - seems to function)

 --- modulename: ipynb_to_batspp, funcname: <module>
ipynb_to_batspp.py(3): import sys
ipynb_to_batspp.py(4): import os
ipynb_to_batspp.py(6): filename_arg = sys.argv
ipynb_to_batspp.py(7): file_in_ipynb = filename_arg[1]
Traceback (most recent call last):
  File "/home/aveey/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/aveey/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/aveey/miniconda3/lib/python3.9/trace.py", line 740, in <module>
    main()
  File "/home/aveey/miniconda3/lib/python3.9/trace.py", line 728, in main
    t.runctx(code, globs, globs)
  File "/home/aveey/miniconda3/lib/python3.9/trace.py", line 450, in runctx
    exec(cmd, globals, locals)
  File "/home/aveey/tom-project/shell-scripts/tests/../tests/ipynb_to_batspp.py", line 7, in <module>
    file_in_ipynb = filename_arg[1]
IndexError: list index out of range
Command exited with non-

: 1

In [38]:
# which-python SHOWS THE PATH OF PYTHON
alias which-python='which python'
which-python
linebr
alias which-py3='which python3' #(ADDED FOR PYTHON3)
which-py3

/home/aveey/miniconda3/bin/python
--------------------------------------------------------------------------------
/home/aveey/miniconda3/bin/python3
